In [1]:
#%pylab
import numpy as np
import matplotlib.pyplot as plt
import random as rdn
mydpi = 200

In [2]:
# tasks列表中存放每个task的type和belief 值
tasks = []
tasks_number = 10000
for i in range(tasks_number):
    realtype = 1 if np.random.uniform()>0.5 else -1
    realbelief = np.random.uniform(0.5,1.0)
    tasks.append([realtype,realbelief])
tasks = np.array(tasks)

# workers数组里存放每个worker的偏离值
workers = []
workers_number = 100
for i in range(workers_number):
    bias = np.random.uniform(-0.2,0.2)
    workers.append(bias)
workers = np.array(workers)

# 每个worker对10000个task的type和belief
workers2tasks = np.zeros((workers_number,tasks_number,2))
for i in range(workers_number):
    for j in range(tasks_number):
        real_pair = tasks[j]
        bias = workers[i]
        if real_pair[1]+bias < 0.5:
            workers2tasks[i,j,0] = 0-real_pair[0]
            workers2tasks[i,j,1] = 1-(real_pair[1]+bias)
        elif real_pair[1]+bias > 1.0:
            workers2tasks[i,j,0] = real_pair[0]
            workers2tasks[i,j,1] = 1.0
        else:
            workers2tasks[i,j,0] = real_pair[0]
            workers2tasks[i,j,1] = real_pair[1]+bias  
            
            
# 三个函数
def r(x,k): # 奖励函数
    c3 = 1.0*(k-1)*2**k/(2**k-k-1)
    c1 = -1.0*(k+1)/(2**k-k-1)
    return -c3*x**k+c3*k/(k-1)*x**(k-1)+c1
def p(x,k): # 惩罚函数
    c3 = 1.0*(k-1)*2**k/(2**k-k-1)
    c2 = -1.0*(k-1)/(2**k-k-1) 
    return c3*x**k+c2
def g(x,k): # 权重函数
    return 1.0*(-2*x*k+(2**k)*(x**k)+k-1)/(2**k-k-1)

In [3]:
Ks = [2,5,9,12]
coeff = [6,7.42857,10.3719,13.105925]
x = np.linspace(0.5,1.0,100)

linestyles = ['-.', '-', '--', ':']
colors = ['b','g','r','c','m','y']

for k in Ks:
    k_index = Ks.index(k)
    y = g(x,k)*coeff[k_index]
    plt.plot(x,y,linewidth=2.5,linestyle = linestyles[k_index],color = colors[k_index],label = str('Personal order value $k$=%d'%k))
plt.plot([0.5,1.0],[4.0/3,8.0/3],linewidth=2.5,color = 'k',label = 'Current crowdsourcing')

plt.xlim(0.5,1.0)
plt.ylabel('Expected gain', fontsize=24)
plt.xlabel("Worker's reporting belief", fontsize=24)
plt.legend(loc='best',scatterpoints=1,ncol=1, fontsize=18,numpoints = 1)
plt.savefig('ExpectedGain.pdf',bbox_inches='tight',dpi=mydpi)
plt.close()

In [4]:
#Performance comparison in latency control between our proposed mechanism and different threshold value settings in  current crowdsourcing

tasks_number = 10000
workers_array = np.arange(100)


thresholds = [0.56,0.62,0.66,0.683]

linestyles = ['-.', '-', '--', ':']
colors = ['b','g','r','c','m','y']

for number  in [3,5]:
    fig = plt.figure()
    ax = fig.gca()
    for i in range(len(thresholds)):
        threshold = thresholds[i]
        tasks_remain = range(tasks_number)
        tasks_status = [0]*tasks_number
        complete_log = []

        while len(tasks_remain)>0:
            task_id = rdn.sample(tasks_remain,1)[0]
            worker_id = rdn.sample(workers_array,1)[0]
            value = tasks[task_id][1] + workers[worker_id]
            if value > threshold or value < 1-threshold:
                tasks_status[task_id] += 1
            if tasks_status[task_id] == number:
                tasks_remain.remove(task_id)
                complete_log.append(1)
            else:
                complete_log.append(0)
        cumsum_complete = np.cumsum(complete_log)
        plt.plot(cumsum_complete,linewidth=2.5,linestyle = linestyles[i],color = colors[i],label = str('Threshold value=%.2f'%threshold))
    plt.plot([0,number*10000],[0,10000],color='k',linewidth=2.5,label = 'Our proposed mechanism')

    plt.legend(loc='best',scatterpoints=1,ncol=1, fontsize=18,numpoints = 1)
    trics = np.arange(0,180000,40000)
    ax.set_xticks(trics)
    ax.set_xlim([0,180000])
    ax.set_ylabel('Number of completed tasks', fontsize=24)
    ax.set_xlabel('Total completion time', fontsize=24)
    ax.xaxis.get_major_formatter().set_powerlimits((-2, 5))  #X轴坐标刻度用科学计数法，坐标取值在10^（-2,5）之外才使用科学计数法
    plt.savefig('Time'+str(number)+'.pdf',bbox_inches='tight',dpi=mydpi)
    plt.close()

In [5]:
np.save('tasks.npy',tasks)
np.save('workers2tasks.npy',workers2tasks)
tasks = np.load('tasks.npy')
workers2tasks = np.load('workers2tasks.npy')

In [6]:
workers_array = np.arange(100)
Ns = [5,7]
Ks = [2,3,5,7,9,10,12] 
It = 30 #求均值和标准差
correct_rate_stats = []
for N in Ns:
    correct_rate_stats.append([])
    majority_correct = 0
    N_index = Ns.index(N)
    for k in Ks:
        correct_Rates = []
        for it in range(It):
            correct_num = 0
            for j in range(tasks_number):
                poslist = rdn.sample(workers_array,N)
                select4task = workers2tasks[poslist,j]
                final4task = 2*(np.dot(g(select4task[:,1],k),select4task[:,0])>0)-1
                if final4task == tasks[j,0]:
                    correct_num += 1
                final_majority = 2*(select4task[:,0].sum()>0)-1
                if final_majority == tasks[j,0]:
                    majority_correct += 1
            correct_rate = 1.0*correct_num/tasks_number
            correct_Rates.append(correct_rate)
        correct_Rates = np.array(correct_Rates)
        correct_rate_stats[N_index].append([correct_Rates.mean(),correct_Rates.std()])
        print correct_Rates.mean(),correct_Rates.std()
    #print '=======================================\n'
    print 1.0*majority_correct/7/(It*tasks_number)
correct_rate_stats = np.array(correct_rate_stats)


linestyles = ['-.', '--', ':']
colors = ['c','r','g','m','y']
markers = ['s','o','d']
for N in Ns:
    N_index = Ns.index(N)
    plt.errorbar(Ks,correct_rate_stats[N_index][:,0],correct_rate_stats[N_index][:,1], linestyle=linestyles[N_index],marker=markers[N_index]
                 ,color=colors[N_index],linewidth=2.5,label = str('%d workers to each task'%(N-2)))
plt.legend(loc='2',scatterpoints=1,ncol=1, fontsize=18,numpoints = 1)
#plt.grid()
plt.ylim(0.94,1.0)
plt.ylabel('Accuracy', fontsize=24)
plt.xlabel('Personal order value', fontsize=24)
plt.xticks(Ks,Ks)
plt.savefig('Accuracy_K_35.pdf',bbox_inches='tight',dpi=mydpi)
#plt.close()

0.972493333333 0.00132411815518
0.972963333333 0.00130524156981
0.973243333333 0.00139372004203
0.973766666667 0.00123782964184
0.974046666667 0.00158087177075
0.974813333333 0.00145733394328
0.97468 0.00139101881128
0.950087619048
0.97929 0.00133724841871
0.98008 0.0013439742061
0.979923333333 0.0012499822221
0.980453333333 0.00122412780742
0.98092 0.000789683480896
0.981053333333 0.00103077101671
0.981026666667 0.0011328234147
0.959256190476


C:\Users\Jerry Xu\Anaconda2\lib\site-packages\matplotlib\legend.py:326: UserWarning: Unrecognized location "2". Falling back on "best"; valid locations are
	right
	center left
	upper right
	lower right
	best
	center
	lower left
	center right
	upper left
	upper center
	lower center

  six.iterkeys(self.codes))))


In [7]:
linestyles = ['-.', '--', ':']
colors = ['c','r','g','m','y']
markers = ['s','o','d']
for N in Ns:
    N_index = Ns.index(N)
    plt.errorbar(Ks,correct_rate_stats[N_index][:,0],correct_rate_stats[N_index][:,1], linestyle=linestyles[N_index],marker=markers[N_index]
                 ,color=colors[N_index],linewidth=2.5,label = str('Our proposed mechanism (%d workers)'%(N-2)))
plt.plot([2,12],[0.943497619048,0.943497619048],color=colors[0],linewidth=2.5,label = str('Simple majority rule (%d workers)'%3))
plt.plot([2,12],[0.952023809524,0.952023809524],color=colors[1],linewidth=2.5,label = str('Simple majority rule (%d workers)'%5))
plt.legend(loc='2',scatterpoints=1,ncol=1, fontsize=18,numpoints = 1)
#plt.grid()
plt.ylim(0.94,1.0)
plt.ylabel('Accuracy', fontsize=24)
plt.xlabel('Personal order value', fontsize=24)
plt.xticks(Ks,Ks)
plt.savefig('Accuracy_K_35.pdf',bbox_inches='tight',dpi=mydpi)
#plt.close()